# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Authenticate" data-toc-modified-id="Authenticate-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Authenticate</a></div><div class="lev2 toc-item"><a href="#what-do-you-need" data-toc-modified-id="what-do-you-need-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>what do you need</a></div><div class="lev1 toc-item"><a href="#defining-helpers" data-toc-modified-id="defining-helpers-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>defining helpers</a></div><div class="lev1 toc-item"><a href="#overview-of-calendars" data-toc-modified-id="overview-of-calendars-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>overview of calendars</a></div><div class="lev1 toc-item"><a href="#push-events-to-calendar" data-toc-modified-id="push-events-to-calendar-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>push events to calendar</a></div><div class="lev2 toc-item"><a href="#read-and-transform-df" data-toc-modified-id="read-and-transform-df-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>read and transform df</a></div>

# Authenticate
Disclaimer: This is really ugly and often fails. Try to get it running, as the apiclient has got the service (this is the hard part), everything should work.

WARNING: oauth2client is outdated and no longer maintained. I still use it but you shouldn't. Unfortunately the [calendar quickstart documentation](https://developers.google.com/calendar/quickstart/python?authuser=1) uses it, so one has to put own effort in using more modern tools.

## what do you need
Just one google account and you can follow the guide [here](https://developers.google.com/calendar/quickstart/python?authuser=1#step_1_turn_on_the_api_name). The normal use of the API is free.


In [1]:
from __future__ import print_function
import httplib2
import os

from apiclient import discovery
from oauth2client import client
from oauth2client import tools
from oauth2client.file import Storage

import datetime

SCOPES = 'https://www.googleapis.com/auth/calendar'
CLIENT_SECRET_FILE = 'client_secret.json'
APPLICATION_NAME = 'Google Calendar API Python Quickstart'

import argparse
from argparse import Namespace

flags = None

In [2]:
home_dir = os.path.expanduser('~')
credential_dir = os.path.join(home_dir, '.credentials')
if not os.path.exists(credential_dir):
    os.makedirs(credential_dir)
credential_path = os.path.join(credential_dir,
                               'calendar-python-quickstart.json')

store = Storage(credential_path)
credentials = store.get()

flow = client.flow_from_clientsecrets(CLIENT_SECRET_FILE, SCOPES)

credentials = tools.run_flow(
    flow, store,
    Namespace(
        auth_host_name='localhost', auth_host_port=[8080, 8090],
        logging_level='ERROR', noauth_local_webserver=False))


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&response_type=code&client_id=781086215716-obdpcfb4v302f9s9lfjtnlph8djdseqa.apps.googleusercontent.com&access_type=offline

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.


In [3]:
http = credentials.authorize(httplib2.Http())

In [4]:
service = discovery.build('calendar', 'v3', credentials=credentials)

# defining helpers

Yes, most of this code is not good. There are no good docstrings nor good code.

This is not a bug, it's an possibility to improve ;-)

Mainly, this section contains of some convenience helpers and two important functions:
- `create_pydata_calendars` (this creates the calendars if there is none with the same summary)
- `write_event` (this creates the event if there is none with same time and summary)

In [5]:
def get_cals(service, filter_list=False):
    """Get Calendar Items from API."""
    cals = service.calendarList().list().execute().get('items')
    if filter_list and isinstance(filter_list, list):
        cals = filter(lambda d: d['summary'] in filter_list, cals)
    return cals


def create_pydata_calendars(service,
                            calendar_list=[
                                'pydata_novice',
                                'pydata_intermediate',
                                'pydata_experienced']):
    """Create Calendars named as `calendar_list`."""
    cals = get_cals(service)
    new_cals = {}
    for summary in calendar_list:
        if summary in map(lambda d: d['summary'], cals):
            continue
        calendar = {'summary': summary, 'timeZone': 'Europe/Berlin'}

        created_calendar = service.calendars().insert(body=calendar).execute()
        new_cals[summary] = created_calendar['id']
    return new_cals


def reduce_to_summary(cals, is_dict=False):
    """Reduce calendar json for summarys or dict of summary:id pairs."""
    if is_dict:
        return {x['summary']: x['id'] for x in cals}
    else:
        return map(lambda d: d['summary'], cals)


def write_event(event, cals, service):
    """Write an event (pd.Series, row of df) to calendar."""
    events = service.events()
    CalID = filter(lambda d: event.level == d['summary'], cals)[0]['id']
    l = (
        events.list(
            calendarId=CalID,
            timeMin=event.start + '+02:00',
            timeMax=event.end + '+02:00',
            singleEvents=True).execute()['items']
    )
    if (len(l) != 0) and any(map(lambda d: d['summary'] == event.summary, l)):
        print("found existing event(s): '{}'".format(
            "', '".join(map(lambda d: d['summary'], l))))
        return False
    else:
        print('creating event {}'.format(event.summary))

    event = {
        'summary': event.summary,
        'location': event.location,
        'description': event.description,
        'start': {'dateTime': event.start, 'timeZone': 'Europe/Berlin'},
        'end': {'dateTime': event.end, 'timeZone': 'Europe/Berlin'},
        # 'organizer': [{'displayName': event.attendees}],  # didn't work
        'reminders': {'useDefault': False}
        # 'attendees': [{'email': 'YOUR@MAIL.com', 'sendNotifications': False}]
    }

    event = events.insert(calendarId=CalID, body=event).execute()
    return True

# overview of calendars

In [6]:
now = datetime.datetime.utcnow().isoformat() + 'Z'
CONFERENCE_START = '2018-07-06'
LEVEL_LIST = ['pydata_novice', 'pydata_intermediate', 'pydata_experienced']

hint: the `LEVEL_LIST` can be created automatically from the dataframe of events later and then execute these functions later -> therefore you can also split by room instead of level if you want to have separate calendars for that.

In [7]:
create_pydata_calendars(service=service)

{'pydata_experienced': u'ms85ehv2i0r01b6v137ks4i3vg@group.calendar.google.com',
 'pydata_intermediate': u't5tb12jnvsscn8h44euot0j1i8@group.calendar.google.com',
 'pydata_novice': u'ig2hfsp67cs1u2tdjplgtpgnkg@group.calendar.google.com'}

In [8]:
reduce_to_summary(get_cals(service, LEVEL_LIST))

[u'pydata_intermediate', u'pydata_novice', u'pydata_experienced']

In [9]:
cals = get_cals(service, LEVEL_LIST)

# push events to calendar
## read and transform df
No big deal: some renamings for the google API, mapping to right date format

In [10]:
df = pd.read_json('./data.json')[
    [
        'description', 'title', 'day_info', 'level',
        'room_inf', 'speaker', 'time_from', 'time_to'
    ]]
df.rename(
    columns={
        'title': 'summary',
        'room_inf': 'location',
        'speaker': 'attendees'
    }, inplace=True)

df.level = 'pydata_' + df.level.str.lower()

# mapping date (dayofweek) to right format for google calendar api values
df['date'] = df.day_info.map(
    {
        ddd.strftime('%A'): ddd.strftime('%Y-%m-%dT')
        for ddd in pd.date_range(
            start=CONFERENCE_START, periods=df.day_info.nunique(), freq="D")})
df['start'] = df['date'] + df.time_from
df['end'] = df['date'] + df.time_to

In [11]:
df.head(2)

,description,summary,day_info,level,location,attendees,time_from,time_to,date,start,end
0,In many practical machine learning classificat...,Using GANs to improve generalization in a semi...,Friday,pydata_intermediate,Track 1,Andreas Merentitis,09:00:00,10:30:00,2018-07-06T,2018-07-06T09:00:00,2018-07-06T10:30:00
1,We’ve created a playground data science applic...,A Hands-On Introduction to Your First Data Sci...,Friday,pydata_intermediate,Track 2,Em Grasmeder,13:15:00,14:45:00,2018-07-06T,2018-07-06T13:15:00,2018-07-06T14:45:00


In [12]:
for (_, event) in df.iterrows():
    write_event(event, cals, service)

creating event Using GANs to improve generalization in a semi-supervised setting - trying it in open datasets
creating event A Hands-On Introduction to Your First Data Science Project
creating event ctparse: a practical parser for natural language time expressions in pure python
creating event Battle-hardened advice on efficient data loading for deep learning on videos.
creating event Where NLP and psychology meet
creating event Towards automating machine learning: benchmarking tools for hyperparameter tuning
creating event Python Unittesting for Ethereum Smart Contracts or how not to create your own Cryptocurrency
creating event LightFields.jl: Fast 3D image reconstruction for VR applications
creating event Object detection to Instance segmentation: Learn to apply several algorithms along the way
creating event Meaningful histogramming with Physt
creating event How to scare a fish (school)
creating event Simple diagrams of convoluted neural networks
creating event Leveling up your sto